# Skillra PDA — Основной отчёт

Этот минимальный ноутбук демонстрирует ключевую витрину `market_view` и служит отправной точкой для HTML-экспорта.

In [ ]:
from pathlib import Path
import sys
import pandas as pd

ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.skillra_pda import config

pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 10)

market_view_path = Path(config.PROCESSED_DATA_DIR) / "market_view.parquet"
market_view_path

In [ ]:
market_view = pd.read_parquet(market_view_path)
print(f"market_view shape: {market_view.shape}")
market_view.head(3)

In [ ]:
summary = (
    market_view
    .select_dtypes("number")
    .agg(["mean", "median", "min", "max"])
)
summary.head(10)